# Matrix to PauliwordOp

We provide three methods of converting a matrix to the `PauliwordOp` representation:

1. Full basis expansion scaling as $\mathcal{O}(4^N)$
2. Expansion over a defined basis $\mathcal{B}$ scaling as $\mathcal{O}(|\mathcal{B}|)$
3. Expansion via projectors scaling as $\mathcal{O}(2^N M)$ where $M$ is the number of non-zero elements

In [1]:
import numpy as np
#from symmer.symplectic.matrix import matrix_to_Pword
from symmer.symplectic import PauliwordOp, get_PauliwordOp_projector
import warnings
warnings.filterwarnings(action='always')

# Full basis method

We can select a basis for any $\mathcal{C}^{2^{N}\times 2^{N}}$ matrix by taking all n-fold tensor product of Pauli operators, which is of size $4^{N}$.

E.g.

$$basis = B = $$
$$\{II, IZ,ZI,ZZ, $$
$$\:IX,IY,ZX,ZY, $$
$$\:XI,XZ,YI,YZ, $$
$$\:XX,XY,YX,YY \}$$

The decomposition of any matrix $M$ is then

$$M = \sum_{P \in basis} c_{i}P_{i}$$

It should be clear that:

$$Trace(M P_{i}) = c_{i} 2^{N}$$

so re-aranging we find:

$$c_{i} = \frac{Trace(M P_{i})}{ 2^{N}}$$

Function currently uses $4^{N}$ terms to build operator...

interesting question is **what are the smallest set of unitaries which we can decompose a given matrix!**

potential solution for CHEMISTRY:

second quantized operator is given as:

$$     H_{e} = \sum_{p=0}^{N-1}  \sum_{q=0}^{N-1} h_{pq} a_{p}^{\dagger} a_{q} + \frac{1}{2} \sum_{p=0}^{N-1}  \sum_{q=0}^{N-1}  \sum_{r=0}^{N-1}  \sum_{s=0}^{N-1}  h_{pqrs} a_{p}^{\dagger} a_{q}^{\dagger} a_{r} a_{s}$$


Therefore basis can be built by getting all the $N^{2}$ 1-RDM operators and all the $N^{4}$ 2-RDM operators  (as Pauli operators!)

TODO: check this out

In [3]:
n_qub = 6
mat = np.arange(2**n_qub * 2**n_qub).reshape([2**n_qub,2**n_qub])
decomp_obj = PauliwordOp.from_matrix(mat, strategy='projector')

Building operator: 100%|██████████████████| 4095/4095 [00:03<00:00, 1108.70it/s]


In [12]:
np.all(decomp_obj.to_sparse_matrix == mat)

True

In [42]:
from scipy.sparse import rand
import numpy as np
from time import time

n_qubits = 10

D = 2**n_qubits
#x = rand(D, D, density=1/(2**(1.9*n_qubits)), format='csr')
x = rand(D, D, density=0.001, format='csr')

In [43]:
p = PauliwordOp.from_matrix(x)
print(p.n_terms)

Building operator: 100%|████████████████████| 1049/1049 [01:49<00:00,  9.59it/s]

657408


In [44]:
p = PauliwordOp.from_matrix(x.toarray())
print(p.n_terms)

Building operator:  75%|███████████████▋     | 786/1049 [01:05<00:21, 11.99it/s]


KeyboardInterrupt: 

In [36]:
1/(2**(1.9*n_qubits))

2.6341780319308768e-09

The user may specify their own pauli basis, however care must be taken to ensure it is sufficiently expressible to represent the input matrix. An error will be thrown if it is not:

In [43]:
P = PauliwordOp.from_dictionary({'XYY':1, 'ZZZ':1})
basis = P[0] + PauliwordOp.from_dictionary({'XIZ':1})
print(basis)
matrix = P.to_sparse_matrix
PauliwordOp.from_matrix(matrix, operator_basis=basis)

 1.000+0.000j XIZ +
 1.000+0.000j XYY


/home/tweaving/anaconda3/envs/symmer/lib/python3.7/site-packages/symmer/symplectic/base.py:184: UserWarning: Basis not sufficiently expressive, output operator projected onto basis supplied.
  warnings.warn('Basis not sufficiently expressive, output operator projected onto basis supplied.')


 1.000+0.000j XYY

The adapted basis now sufficiently expressive, and note redundancy in the basis is okay. Defining a basis can circumvent accessing the full $4^N$-dimensional Hilbert space of $2^N \times 2^N$ square matrices.

In [6]:
P = PauliwordOp.from_dictionary({'XYY':1, 'ZZZ':1})
basis = P + PauliwordOp.from_dictionary({'XIZ':1})
print(basis)
matrix = P.to_sparse_matrix.todense()
PauliwordOp.from_matrix(matrix, operator_basis=basis)

 1.000+0.000j ZZZ +
 1.000+0.000j XIZ +
 1.000+0.000j XYY


 1.000+0.000j ZZZ +
 1.000+0.000j XYY

# Projector method

This strategy uses projectors on the computational basis to pick non-zero matrix elements out. In particular, we may write $$ | 0 \rangle \langle 0 | = \sum_{\vec{Z} \in \{I, Z\}^{\otimes N}} \vec{Z}.$$ An arbitrary projection $| i \rangle \langle j |$ may be obtained via application of Pauli $X$ operators on either side of the base projection, $$| i \rangle \langle j | = \vec{X}_i | 0 \rangle \langle 0 | \vec{X}_j.$$ For example,

In [17]:
_00_projector = get_PauliwordOp_projector('00')
_00_projector.to_sparse_matrix.toarray().real

array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [24]:
# change row and column index for two qubit example
i=1
j=2


Xi = np.binary_repr(i, width=_00_projector.n_qubits)
Xj = np.binary_repr(j, width=_00_projector.n_qubits)
Xi_op = PauliwordOp.from_list([Xi.replace('0', 'I').replace('1', 'X')])
Xj_op = PauliwordOp.from_list([Xj.replace('0', 'I').replace('1', 'X')])
_ij_projector = (Xi_op * _00_projector * Xj_op)
_ij_projector.to_sparse_matrix.toarray().real

array([[0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [8]:
# basis supplied
%timeit PauliwordOp.from_matrix(matrix, operator_basis=basis, strategy='full_basis')

# no basis supplied
%timeit PauliwordOp.from_matrix(matrix, strategy='full_basis')

# projector method
%timeit PauliwordOp.from_matrix(matrix, strategy='projector')

1.55 ms ± 36.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
24.1 ms ± 1.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
6.18 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
from scipy.sparse import csr_matrix
nq = 20

row = np.array([1, 1000])
col = np.array([2000,478])

data = np.array([1e6,1e3])

sparse_mat = csr_matrix((data, (row, col)), shape=(2**nq, 2**nq))

In [34]:
P_sparse = PauliwordOp.from_matrix(sparse_mat, strategy='projector')
P_sparse.n_terms

2097152